<a href="https://colab.research.google.com/github/ag497/Black-Scholes-Merlton-/blob/main/BSM_European_Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#IMPORTING NECESSARY LIBRARIES
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm
from math import sqrt, log, pow, erf, e

In [6]:
#GETTING TICKER
ticker='CRM'
option=yf.Ticker(ticker)
option.options

('2024-06-07',
 '2024-06-14',
 '2024-06-21',
 '2024-06-28',
 '2024-07-05',
 '2024-07-12',
 '2024-07-19',
 '2024-08-16',
 '2024-09-20',
 '2024-10-18',
 '2024-11-15',
 '2024-12-20',
 '2025-01-17',
 '2025-03-21',
 '2025-06-20',
 '2026-01-16',
 '2026-06-18',
 '2026-12-18')

In [7]:
#FETCH CALL PUT DATA
date='2024-12-20'
def fetch_option_data(ticker_symbol,date):
  ticker=yf.Ticker(ticker_symbol)
  options_data=ticker.option_chain(date)
  return options_data.calls, options_data.puts

calls, puts = fetch_option_data(ticker, date)



In [8]:
#CALLS DATAFRAME
calls = pd.DataFrame(calls)
calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,CRM241220C00140000,2024-05-30 13:38:15+00:00,140.0,85.9,0.0,0.0,0.0,0.0,1,0,0.00001,True,REGULAR,USD
1,CRM241220C00150000,2024-05-31 13:30:20+00:00,150.0,75.5,0.0,0.0,0.0,0.0,3,0,0.00001,True,REGULAR,USD
2,CRM241220C00160000,2024-05-30 19:37:34+00:00,160.0,62.3,0.0,0.0,0.0,0.0,28,0,0.00001,True,REGULAR,USD
3,CRM241220C00165000,2024-05-30 17:35:27+00:00,165.0,56.9,0.0,0.0,0.0,0.0,1,0,0.00001,True,REGULAR,USD
4,CRM241220C00170000,2024-05-30 19:30:43+00:00,170.0,54.0,0.0,0.0,0.0,0.0,8,0,0.00001,True,REGULAR,USD


In [9]:
#PUT DATA FRAME
puts = pd.DataFrame(puts)
puts.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,CRM241220P00135000,2024-05-31 19:59:33+00:00,135.0,0.59,0.0,0.0,0.0,0.0,4.0,0,0.125009,False,REGULAR,USD
1,CRM241220P00140000,2024-06-03 19:51:48+00:00,140.0,0.71,0.0,0.0,0.0,0.0,1.0,0,0.125009,False,REGULAR,USD
2,CRM241220P00145000,2024-05-31 17:24:11+00:00,145.0,1.10,0.0,0.0,0.0,0.0,15.0,0,0.125009,False,REGULAR,USD
3,CRM241220P00150000,2024-06-03 19:19:03+00:00,150.0,1.10,0.0,0.0,0.0,0.0,3.0,0,0.125009,False,REGULAR,USD
4,CRM241220P00155000,2024-05-31 15:05:02+00:00,155.0,2.17,0.0,0.0,0.0,0.0,1.0,0,0.125009,False,REGULAR,USD


In [10]:
#BSM STANDARD OPERATIONS
def standard_normal_cdf(x):
    return 0.5 * (1 + erf(x / sqrt(2)))

def d1(S, K, t, r, vol):
    numerator = log(S/K) + (r + pow(vol, 2)/2) * t
    denominator = vol * sqrt(t)
    return numerator / denominator

def d2(S, K, t, r, vol):
    return d1(S, K, t, r, vol) - vol * sqrt(t)

def call_price(S, K, t, r, vol):
    Nd1 = standard_normal_cdf(d1(S, K, t, r, vol))
    Nd2 = standard_normal_cdf(d2(S, K, t, r, vol))
    return Nd1 * S - Nd2 * K * pow(e, -1 * r * t)

def put_price(S, K, t, r, vol):
    Nd1 = standard_normal_cdf(- 1 * d1(S, K, t, r, vol))
    Nd2 = standard_normal_cdf(- 1 * d2(S, K, t, r, vol))
    return Nd2 * K * pow(e, -1 * r * t) - S * Nd1

In [11]:
#CREATING DATAFRAME FOR BSM AND RELATED
def get_current_stock_price(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    current_price = stock.history(period='1d')['Close'].iloc[-1]
    return current_price



def get_10yr_treasury_rate():
    # 10 year treasury ticker symbol
    treasury_ticker = "^TNX"

    now = datetime.now()
    ten_years_ago = now.replace(year=now.year - 10)

    treasury_data = yf.download(treasury_ticker, start=ten_years_ago, end=now)
    last_yield = treasury_data['Close'].iloc[-1]
    return last_yield



def get_date(date_str):
    # date string is in the format 'YYYY-MM-DD'
    date_object = datetime.strptime(date_str, '%Y-%m-%d')

    current_date = datetime.now()
    years_difference = (date_object - current_date).days / 365.25
    return years_difference


#ANNUAL VOLATILITY CALCULATION
def calculate_volatility():

    today = datetime.now()
    one_year_ago = today.replace(year=today.year-1)
    data = yf.download(ticker, start=one_year_ago, end=today)

    # Calculate daily returns
    data['Daily_Return'] = data['Adj Close'].pct_change()

    # std of daily returns
    daily_volatility = data['Daily_Return'].std()
    annualized_volatility = daily_volatility * np.sqrt(252)

    return annualized_volatility



S = get_current_stock_price('CRM')
r = get_10yr_treasury_rate()
t = get_date('2024-10-28')
vol = calculate_volatility()

# Converting r to percentage
r = r / 100

print(f"Riskfree rate (in percentage) is: {r}")
print(f"Years difference is : {t}")
print(f"The current stock price of {ticker} is: {S}")
print(f"Stock price of {ticker}: {S}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Riskfree rate: 0.04401999950408936
Years difference is : 0.39698836413415467
The current stock price of CRM is: 236.6199951171875
Stock price of CRM: 236.6199951171875


In [47]:
main_df = calls.copy()
columns_to_drop = ['lastTradeDate', 'lastPrice', 'volume', 'openInterest', 'contractSize', 'currency']
main_df.drop(columns=columns_to_drop, inplace=True)
main_df['BSM Valuation'] = main_df.apply(lambda row: call_price(S, row['strike'], t, r, vol), axis=1)
puts_df =puts.copy()
columns_to_drop = ['lastTradeDate', 'lastPrice', 'volume', 'openInterest', 'contractSize', 'currency']
puts_df.drop(columns=columns_to_drop, inplace=True)
puts_df['BSM Valuation'] = puts_df.apply(lambda row: put_price(S, row['strike'], t, r, vol), axis=1)

In [44]:
#GREEKS

def d1(S, K, t, r, vol):
    return (log(S / K) + (r + 0.5 * vol ** 2) * t) / (vol * sqrt(t))

def d2(S, K, t, r, vol):
    return d1(S, K, t, r, vol) - vol * sqrt(t)

def delta(S, K, t, r, vol, is_call=True):
    if is_call:
        return norm.cdf(d1(S, K, t, r, vol))
    else:
        return norm.cdf(d1(S, K, t, r, vol)) - 1

def gamma(S, K, t, r, vol,is_call=True):
    d1_value = d1(S, K, t, r, vol)
    if is_call:
      numerator = norm.pdf(d1_value)
    else:
      numerator = norm.pdf(d1_value) * exp(-r * t)
    denominator = S * vol * sqrt(exp(1))
    return numerator / denominator


def vega(S, K, t, r, vol, is_call=True):
    d1_value = d1(S, K, t, r, vol)
    if is_call:
        return S * norm.pdf(d1_value) * sqrt(t)
    else:
        return S * norm.pdf(d1_value) * sqrt(t)


def theta(S, K, t, r, vol, is_call=True):
    numerator = S * norm.pdf(d1(S, K, t, r, vol)) * vol
    denominator = 2 * sqrt(t)
    if is_call:
        return -1 * (numerator / denominator) - r * K * exp(-1 * r * t) * norm.cdf(d2(S, K, t, r, vol))
    else:
        return -1 * (numerator / denominator) + r * K * exp(-1 * r * t) * norm.cdf(-1 * d2(S, K, t, r, vol))

def rho(S, K, t, r, vol, is_call=True):
    if is_call:
        return K * t * exp(-1 * r * t) * norm.cdf(d2(S, K, t, r, vol))
    else:
        return -1 * K * t * exp(-1 * r * t) * norm.cdf(-1 * d2(S, K, t, r, vol))


In [51]:
#SECOND ORDER GREEKS
def vanna(S, K, t, r, vol, is_call=True):
    d1_value = d1(S, K, t, r, vol)
    vega_value = vega(S, K, t, r, vol)
    if is_call:
        return -(norm.pdf(d1_value) / vol) * (d1_value / S) * sqrt(t)
    else:
        return (norm.pdf(d1_value) / vol) * (d1_value / S) * sqrt(t)

def charm(S, K, t, r, vol, is_call=True):
    d1_value = d1(S, K, t, r, vol)
    if is_call:
        charm_value = -(norm.pdf(d1_value) * vol) / (2 * sqrt(t)) + (r * norm.cdf(d1_value) - (vol / (2 * sqrt(t)))) * delta(S, K, t, r, vol)
    else:
        charm_value = -(norm.pdf(d1_value) * vol) / (2 * sqrt(t)) - (r * norm.cdf(-d1_value) - (vol / (2 * sqrt(t)))) * delta(S, K, t, r, vol)
    return charm_value

def volga(S, K, t, r, vol, is_call=True):
    d1_value = d1(S, K, t, r, vol)
    vega_value = vega(S, K, t, r, vol)
    if is_call:
        return S * norm.pdf(d1_value) * sqrt(t) * d1_value * d2(S, K, t, r, vol) / vol
    else:
        return -S * norm.pdf(d1_value) * sqrt(t) * d1_value * d2(S, K, t, r, vol) / vol

def veta(S, K, t, r, vol, is_call=True):
    d1_value = d1(S, K, t, r, vol)
    if is_call:
        return -S * norm.pdf(d1_value) * sqrt(t) * r / (2 * vol) + 0.5 * S * norm.pdf(d1_value) * vol * sqrt(t) / t
    else:
        return -S * norm.pdf(d1_value) * sqrt(t) * r / (2 * vol) - 0.5 * S * norm.pdf(d1_value) * vol * sqrt(t) / t

In [49]:

greeks_call_df = main_df.copy()

# For call options
greeks_call_df['CALL DELTA'] = greeks_call_df.apply(lambda row: delta(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL GAMMA'] = greeks_call_df.apply(lambda row: gamma(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL VEGA'] = greeks_call_df.apply(lambda row: vega(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL THETA'] = greeks_call_df.apply(lambda row: theta(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL RHO'] = greeks_call_df.apply(lambda row: rho(S, row['strike'], t, r, vol), axis=1)
# For second order greeks
greeks_call_df['CALL VANNA'] = greeks_call_df.apply(lambda row: vanna(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL CHARM'] = greeks_call_df.apply(lambda row: charm(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL VOLGA'] = greeks_call_df.apply(lambda row: volga(S, row['strike'], t, r, vol), axis=1)
greeks_call_df['CALL VETA'] = greeks_call_df.apply(lambda row: veta(S, row['strike'], t, r, vol), axis=1)

greeks_call_df.head()



,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,BSM Valuation,CALL DELTA,CALL GAMMA,CALL VEGA,CALL THETA,CALL RHO,CALL VANNA,CALL CHARM,CALL VOLGA,CALL VETA
0,CRM241220C00140000,140.0,0.0,0.0,0.0,0.0,0.00001,True,99.113280,0.995929,0.000091,1.794101,-6.776511,54.206199,-0.000250,-0.227443,34.074052,0.649356
1,CRM241220C00150000,150.0,0.0,0.0,0.0,0.0,0.00001,True,89.403728,0.989913,0.000203,4.003719,-8.084495,57.495599,-0.000490,-0.230337,57.890043,1.449105
2,CRM241220C00160000,160.0,0.0,0.0,0.0,0.0,0.00001,True,79.828756,0.978353,0.000392,7.718584,-9.971381,60.210924,-0.000822,-0.234932,83.176226,2.793662
3,CRM241220C00165000,165.0,0.0,0.0,0.0,0.0,0.00001,True,75.119338,0.969726,0.000518,10.221027,-11.157063,61.270050,-0.001011,-0.237865,94.133866,3.699396
4,CRM241220C00170000,170.0,0.0,0.0,0.0,0.0,0.00001,True,70.479767,0.958805,0.000667,13.158088,-12.501316,62.086067,-0.001204,-0.241154,102.730470,4.762434


In [50]:
greeks_puts_df=puts_df.copy()

# For put options
greeks_puts_df['PUT DELTA'] = greeks_puts_df.apply(lambda row: delta(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT GAMMA'] = greeks_puts_df.apply(lambda row: gamma(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT VEGA'] = greeks_puts_df.apply(lambda row: vega(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT THETA'] = greeks_puts_df.apply(lambda row: theta(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT RHO'] = greeks_puts_df.apply(lambda row: rho(S, row['strike'], t, r, vol, is_call=False), axis=1)
# For second order greeks
greeks_puts_df['PUT VANNA'] = greeks_puts_df.apply(lambda row: vanna(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT CHARM'] = greeks_puts_df.apply(lambda row: charm(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT VOLGA'] = greeks_puts_df.apply(lambda row: volga(S, row['strike'], t, r, vol, is_call=False), axis=1)
greeks_puts_df['PUT VETA'] = greeks_puts_df.apply(lambda row: veta(S, row['strike'], t, r, vol, is_call=False), axis=1)

greeks_puts_df.head()


,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,BSM Valuation,PUT DELTA,PUT GAMMA,PUT VEGA,PUT THETA,PUT RHO,PUT VANNA,PUT CHARM,PUT VOLGA,PUT VETA
0,CRM241220P00135000,135.0,0.0,0.0,0.0,0.0,0.125009,False,0.038670,-0.002428,0.000056,1.126779,-0.454008,-0.243407,0.000167,0.266171,-24.371898,-0.554170
1,CRM241220P00140000,140.0,0.0,0.0,0.0,0.0,0.125009,False,0.067978,-0.004071,0.000089,1.794101,-0.720473,-0.409353,0.000250,0.264454,-34.074052,-0.882372
2,CRM241220P00145000,145.0,0.0,0.0,0.0,0.0,0.125009,False,0.114430,-0.006535,0.000136,2.734172,-1.094051,-0.659323,0.000358,0.261987,-45.413537,-1.344716
3,CRM241220P00150000,150.0,0.0,0.0,0.0,0.0,0.125009,False,0.185190,-0.010087,0.000200,4.003719,-1.595883,-1.021064,0.000490,0.258588,-57.890043,-1.969102
4,CRM241220P00155000,155.0,0.0,0.0,0.0,0.0,0.125009,False,0.289159,-0.015021,0.000282,5.652713,-2.243839,-1.525774,0.000646,0.254074,-70.774939,-2.780107


In [ ]:
#references:
#https://medium.com/@SSiddhant/implementing-black-scholes-model-for-option-chains-using-python-6978662bfbb6
#https://medium.com/hypervolatility/options-greeks-vanna-charm-vomma-dvegadtime-77d35c4db85c